# Восстановление пробелов в тексте

Подготовим импорты

In [ ]:
import os
import glob
from bs4 import BeautifulSoup
import json
import random
import pandas as pd
from razdel import sentenize
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification
from torch.optim import AdamW 
from sklearn.metrics import f1_score, precision_score, recall_score
from tqdm import tqdm

Создание корпуса для обучения

In [42]:
input_folder = "news_raw/texts"
output_file = "corpus.txt"

all_files = glob.glob(os.path.join(input_folder, "**/*.txt"), recursive=True)
lines = []

for f in all_files:
    with open(f, "r", encoding="utf-8") as file:
        html = file.read()
        soup = BeautifulSoup(html, "html.parser")

        # Удаляем script и style
        for tag in soup(["script", "style"]):
            tag.decompose()

        text = soup.get_text(separator=" ", strip=True)

        # Убираем лишние символы переноса строк
        text = text.replace('\u2028',' ').replace('\u2029',' ')

        # Сжимаем множественные пробелы в один
        text = " ".join(text.split())

        if text:
            lines.append(text)

with open(output_file, "w", encoding="utf-8") as f:
    for line in lines:
        f.write(line + "\n")

print("corpus.txt готов! Всего строк:", len(lines))

corpus.txt готов! Всего строк: 36445


Подготовим корпус к обучению

In [ ]:
def positions_from_text(s):
    """Возвращает индексы пробелов в строке"""
    return [i+1 for i, c in enumerate(s[:-1]) if s[i+1].isspace()]

def remove_spaces(s):
    """Удаляет пробелы"""
    return s.replace(" ", "")

def extract_sentences_from_file(filename):
    """Читает файл и разбивает на предложения с помощью razdel"""
    sentences = []
    with open(filename, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            for sent in sentenize(line):
                text = sent.text.strip()
                if len(text) > 3:  # минимальная длина предложения
                    sentences.append(text)
    return sentences

def generate_synthetic_dataset(sentences, seed=42, val_size=0.1, test_size=0.1):
    random.seed(seed)
    data = []
    for i, line in enumerate(sentences):
        line_norm = " ".join(line.split())  # убираем лишние пробелы
        no_space = remove_spaces(line_norm)
        positions = positions_from_text(line_norm)
        data.append({
            "id": i,
            "text": line_norm,
            "no_space": no_space,
            "space_positions": json.dumps(positions)
        })

    df = pd.DataFrame(data)
    # Делим на train/val/test
    train_df, temp_df = train_test_split(df, test_size=val_size+test_size, random_state=seed)
    val_ratio = val_size / (val_size + test_size)
    val_df, test_df = train_test_split(temp_df, test_size=1-val_ratio, random_state=seed)

    return train_df, val_df, test_df

In [ ]:
corpus_file = "corpus.txt"  
if not os.path.exists(corpus_file):
    raise FileNotFoundError(f"Файл {corpus_file} не найден!")

print("Извлекаем предложения...")
sentences = extract_sentences_from_file(corpus_file)
print(f"Всего предложений: {len(sentences)}")

print("Генерируем синтетический датасет...")
train_df, val_df, test_df = generate_synthetic_dataset(sentences)
os.makedirs("data", exist_ok=True)
train_df.to_csv("data/train.csv", index=False)
val_df.to_csv("data/val.csv", index=False)
test_df.to_csv("data/test.csv", index=False)

print("Синтетический датасет создан!")
print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

Извлекаем предложения...
Всего предложений: 293223
Генерируем синтетический датасет...
Синтетический датасет создан!
Train: 234578, Val: 29322, Test: 29323


Так как данные из новостей, они могут пересекаться, проверим и удалим дубликаты во всех датасетах

In [45]:
def check_dataset_overlap(train_file, val_file, test_file):
    train_df = pd.read_csv(train_file)
    val_df = pd.read_csv(val_file)
    test_df = pd.read_csv(test_file)

    datasets = {"train": train_df, "val": val_df, "test": test_df}
    pairs = [("train", "val"), ("train", "test"), ("val", "test")]

    for a, b in pairs:
        df_a, df_b = datasets[a], datasets[b]

        id_overlap = set(df_a['id']).intersection(df_b['id'])
        text_overlap = set(df_a['text']).intersection(df_b['text'])

        print(f"\nПересечение {a} & {b}:")
        print(f"  ID пересекаются: {len(id_overlap)}")
        print(f"  Text пересекаются: {len(text_overlap)}")

check_dataset_overlap(
    "data/train.csv",
    "data/val.csv",
    "data/test.csv"
)


Пересечение train & val:
  ID пересекаются: 0
  Text пересекаются: 595

Пересечение train & test:
  ID пересекаются: 0
  Text пересекаются: 603

Пересечение val & test:
  ID пересекаются: 0
  Text пересекаются: 130


In [46]:
val_df = val_df[~val_df['text'].isin(train_df['text'])]
test_df = test_df[~test_df['text'].isin(train_df['text'])]
test_df = test_df[~test_df['text'].isin(val_df['text'])]

val_df.to_csv("data/val.csv", index=False)
test_df.to_csv("data/test.csv", index=False)

print("Пересечения удалены. Новые размеры:")
print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

Пересечения удалены. Новые размеры:
Train: 234578, Val: 28628, Test: 28568


Подготовка к обучению

In [50]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

MAX_LEN = 128    
BATCH_SIZE = 16
EPOCHS = 50

CHECKPOINT_DIR = "./checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

Using device: cuda


In [49]:
train_df = pd.read_csv("data/train.csv")
val_df   = pd.read_csv("data/val.csv")
test_df  = pd.read_csv("data/test.csv") 

In [58]:
train_df.sample(20)

,id,text,no_space,space_positions
8135,94411,"Последний из них, ""Dig Out Your Soul"", поступи...","Последнийизних,""DigOutYourSoul"",поступилвпрода...","[9, 12, 17, 22, 26, 31, 38, 47, 49, 57, 64, 69]"
151,283878,«Сирию раздирают войны и конфликты.,«Сириюраздираютвойныиконфликты.,"[6, 16, 22, 24]"
49394,474,"Сообщается, что клубы могут заключить сделку в...","Сообщается,чтоклубымогутзаключитьсделкувближай...","[11, 15, 21, 27, 37, 44, 46, 56]"
121378,87546,В России работают более 270 закусочных McDonal...,ВРоссииработаютболее270закусочныхMcDonald's.,"[1, 8, 17, 23, 27, 38]"
38025,7215,Дату проверки источник не указал.,Датупроверкиисточникнеуказал.,"[4, 13, 22, 25]"
160285,190910,Пророссийские митинги также прошли в других ре...,Пророссийскиемитингитакжепрошливдругихрегионах...,"[13, 21, 27, 34, 36, 43, 52, 56, 58, 66]"
174591,235977,Миссия осуждает такие действия.,Миссияосуждаеттакиедействия.,"[6, 15, 21]"
3284,125190,В каком состоянии находится животное и как себ...,Вкакомсостояниинаходитсяживотноеикаксебячувств...,"[1, 7, 17, 27, 36, 38, 42, 47, 58, 61]"
185799,289613,Спустя пять лет он был переизбран на второй срок.,Спустяпятьлетонбылпереизбраннавторойсрок.,"[6, 11, 15, 18, 22, 33, 36, 43]"
122675,222433,Всего с платформы эвакуированы около 300 работ...,Всегосплатформыэвакуированыоколо300работников.,"[5, 7, 17, 30, 36, 40]"


Так как в тестовом файле от Авито в основном короткие словосочетания, такие как книгахорошая, оставим из исходного датасета только короткие строки

In [51]:
train_df = train_df[train_df['no_space'].str.len() <= MAX_LEN/2].copy()
val_df = val_df[val_df['no_space'].str.len() <= MAX_LEN/2].copy()
test_df = test_df[test_df['no_space'].str.len() <= MAX_LEN/2].copy()
print(f"Количество коротких строк: {len(train_df)}")
print(f"Количество коротких строк: {len(val_df)}")
print(f"Количество коротких строк: {len(test_df)}")

Количество коротких строк: 36160
Количество коротких строк: 4285
Количество коротких строк: 4023


In [59]:
train_df.sample(20)

,id,text,no_space,space_positions
228066,258970,«Наши спортсмены давно не употребляют запрещен...,«Нашиспортсменыдавнонеупотребляютзапрещенныхпр...,"[5, 16, 22, 25, 37, 49]"
21730,153928,"Кроме того, в аварии могли пострадать водитель...","Крометого,вавариимоглипострадатьводительипасса...","[5, 11, 13, 20, 26, 37, 46, 48, 58]"
194526,168136,"В числе последних назывались «ЮТэйр», «Сибирь»...","Вчислепоследнихназывались«ЮТэйр»,«Сибирь»(S7)и...","[1, 7, 17, 28, 37, 46, 51, 53]"
32380,267288,Вместо этого руководство ЦБ сделало все правил...,"ВместоэтогоруководствоЦБсделаловсеправильно»,—...","[6, 12, 24, 27, 35, 39, 51, 53, 61]"
71417,108723,В 2010 году она отобрала это звание у Microsoft.,В2010годуонаотобралаэтозваниеуMicrosoft.,"[1, 6, 11, 15, 24, 28, 35, 37]"
216410,57061,29 августа 2010 года группа выступила в москов...,29августа2010годагруппавыступилавмосковскомклу...,"[2, 10, 15, 20, 27, 37, 39, 50, 56]"
126138,122764,В первое семейство вошли модели с экранами в 1...,"Впервоесемействовошлимоделисэкранамив11,6,14и1...","[1, 8, 18, 24, 31, 33, 42, 44, 50, 53, 55, 60]"
21128,126568,"На первом месте ""Реал"", у которого 78 очков и ...","Напервомместе""Реал"",укоторого78очковиоднаиграв...","[2, 9, 15, 23, 25, 34, 37, 43, 45, 50, 55, 57]"
205125,105518,"О том, когда выйдет Dragon Quest X для Wii U, ...","Отом,когдавыйдетDragonQuestXдляWiiU,поканесооб...","[1, 6, 12, 19, 26, 32, 34, 38, 42, 45, 50, 53]"
46819,186844,Всего Следственным департаментом МВД задержано...,ВсегоСледственнымдепартаментомМВДзадержано12че...,"[5, 18, 32, 36, 46, 49]"


Подготовка данных и датасета для BERT

In [52]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class SpaceDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=MAX_LEN):
        self.texts = df['no_space'].tolist()
        self.labels = []
        for pos_str, txt in zip(df['space_positions'], df['no_space']):
            pos = [int(p) for p in pos_str.strip("[]").split(",") if p]
            label = [0]*len(txt)
            for p in pos:
                if p < len(label):
                    label[p] = 1
            if len(label) > max_len:
                label = label[:max_len]
            self.labels.append(label)

        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        txt = self.texts[idx]
        label = self.labels[idx]
        enc = self.tokenizer(txt, truncation=True, max_length=self.max_len, padding='max_length', return_tensors='pt')
        input_ids = enc['input_ids'].squeeze(0)
        attention_mask = enc['attention_mask'].squeeze(0)
        labels = torch.tensor(label[:self.max_len], dtype=torch.long)
        if len(labels) < self.max_len:
            pad_len = self.max_len - len(labels)
            labels = torch.cat([labels, torch.zeros(pad_len, dtype=torch.long)])
        return input_ids, attention_mask, labels

train_dataset = SpaceDataset(train_df, tokenizer)
val_dataset   = SpaceDataset(val_df, tokenizer)
train_loader  = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader    = DataLoader(val_dataset, batch_size=BATCH_SIZE)

Архитектура и инициализация модели

In [53]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [54]:
optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = nn.CrossEntropyLoss()

Функции для обучения и валидации

In [55]:
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for input_ids, attention_mask, labels in tqdm(loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
        loss = criterion(outputs.view(-1, 2), labels.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for input_ids, attention_mask, labels in loader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
            loss = criterion(outputs.view(-1, 2), labels.view(-1))
            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)
            mask = attention_mask.bool()
            all_preds.extend(preds[mask].cpu().numpy())
            all_labels.extend(labels[mask].cpu().numpy())

    f1 = f1_score(all_labels, all_preds)
    p = precision_score(all_labels, all_preds)
    r = recall_score(all_labels, all_preds)
    return total_loss / len(loader), f1, p, r


Обучение модели

In [ ]:
best_f1 = 0
epochs_no_improve = 0
patience = 7
min_delta = 0.001

for epoch in range(EPOCHS):
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, f1, p, r = validate(model, val_loader, criterion, device)
    
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, F1={f1:.4f}, P={p:.4f}, R={r:.4f}")

    # Проверка на улучшение F1
    if f1 - best_f1 > min_delta:
        best_f1 = f1
        epochs_no_improve = 0
        torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, "best_epoch_5.pt"))
        print("Saved best checkpoint.")
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print(f"No improvement for {patience} epochs. Stopping training.")
        break

    # уменьшение learning rate, если нет улучшений
    if epochs_no_improve > 0 and epochs_no_improve % 2 == 0:
        for g in optimizer.param_groups:
            g['lr'] *= 0.5
        print(f"Learning rate reduced to {optimizer.param_groups[0]['lr']:.6f}")

100%|██████████| 2260/2260 [04:14<00:00,  8.89it/s]


Epoch 1: Train Loss=0.1175, Val Loss=0.0943, F1=0.4697, P=0.8176, R=0.3295
Saved best checkpoint.


100%|██████████| 2260/2260 [04:11<00:00,  8.97it/s]


Epoch 2: Train Loss=0.0883, Val Loss=0.0776, F1=0.6048, P=0.8208, R=0.4788
Saved best checkpoint.


100%|██████████| 2260/2260 [04:18<00:00,  8.75it/s]


...

Epoch 19: Train Loss=0.0135, Val Loss=0.0589, F1=0.8718, P=0.8835, R=0.8603


100%|██████████| 2260/2260 [04:11<00:00,  8.98it/s]


Epoch 20: Train Loss=0.0128, Val Loss=0.0616, F1=0.8743, P=0.8821, R=0.8666
Saved best checkpoint.


 44%|████▍     | 997/2260 [01:53<02:23,  8.80it/s]


KeyboardInterrupt: 

Функция для проверки метрики на тестовом датасете

In [ ]:
def evaluate_file(model, tokenizer, df, max_len=MAX_LEN):
    if 'no_space' not in df.columns:
        df['no_space'] = df['tetext_no_spaces'] 
    if 'space_positions' not in df.columns and 'true_positions' in df.columns:
        df['space_positions'] = df['true_positions']

    dataset = SpaceDataset(df, tokenizer, max_len=max_len)
    loader = DataLoader(dataset, batch_size=16)
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for input_ids, attention_mask, labels in loader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
            preds = torch.argmax(outputs, dim=-1)

            mask = attention_mask.bool()
            all_preds.extend(preds[mask].cpu().numpy())
            all_labels.extend(labels[mask].cpu().numpy())

    f1 = f1_score(all_labels, all_preds)
    p = precision_score(all_labels, all_preds)
    r = recall_score(all_labels, all_preds)
    print(f"Evaluation: F1={f1:.4f}, Precision={p:.4f}, Recall={r:.4f}")

In [61]:
best_model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=2)
best_model.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, "best_epoch_5.pt"), map_location=device))
best_model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [62]:
evaluate_file(best_model, tokenizer, test_df)

Evaluation: F1=0.8725, Precision=0.8794, Recall=0.8657


Обработка файла задания от Авито

In [ ]:
with open("dataset_1937770_3.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# Заменяем первую запятую в каждой строке на точку с запятой
corrected_lines = []
for line in lines:
    # Разделяем строку по первой запятой
    parts = line.split(",", 1)
    if len(parts) == 2:
        # Объединяем с разделителем ';'
        corrected_line = f"{parts[0]};{parts[1]}"
        corrected_lines.append(corrected_line)
    else:
        # Если строка не содержит запятых (например, заголовок), оставляем как есть
        corrected_lines.append(line)

with open("corrected_dataset.txt", "w", encoding="utf-8") as file:
    file.writelines(corrected_lines)

task_data = pd.read_csv("corrected_dataset.txt", sep=";", encoding="utf-8")

In [34]:
task_data

,id,text_no_spaces
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку
...,...,...
1000,1000,Янеусну.
1001,1001,Весна-яуженегреюпио.
1002,1002,Весна-скоровырастеттрава.
1003,1003,"Весна-выпосмотрите,каккрасиво."


Подготовка датасета для инференса

In [35]:
class TaskDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.texts = df['text_no_spaces'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        txt = self.texts[idx]
        enc = self.tokenizer(
            txt, truncation=True, max_length=self.max_len,
            padding='max_length', return_tensors='pt'
        )
        input_ids = enc['input_ids'].squeeze(0)
        attention_mask = enc['attention_mask'].squeeze(0)
        return input_ids, attention_mask, txt

task_dataset = TaskDataset(task_data, tokenizer, MAX_LEN)
task_loader = DataLoader(task_dataset, batch_size=BATCH_SIZE)

Загрузка сохраненной модели и подготовка ее к инференсу

In [38]:
model.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, "best_epoch_5.pt")))
model.to(device)
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

Инференс модели и восстановление текста с пробелами

In [ ]:
pred_positions = []
pred_texts = []

with torch.no_grad():
    for input_ids, attention_mask, texts in task_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
        preds = torch.argmax(outputs, dim=-1)

        for i, txt in enumerate(texts):
            mask = attention_mask[i].bool()
            pred_labels = preds[i][mask].cpu().numpy()
            positions = [j for j, label in enumerate(pred_labels) if label == 1]

            # Восстанавливаем текст с пробелами
            new_text = ""
            for idx, char in enumerate(txt):
                if idx in positions:
                    new_text += " "
                new_text += char

            pred_positions.append(positions)
            pred_texts.append(new_text)

task_data['predicted_positions'] = pred_positions
task_data['predicted_text'] = pred_texts

task_data.to_csv("submission.csv", index=False)
print("Predictions added to task_data and saved to submission.csv")

Predictions added to task_data and saved to submission.csv
